<a href="https://colab.research.google.com/github/Anubhav1107/First_One/blob/master/Inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch
from torch import nn
from torchvision import datasets, transforms


transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.5,),(0.5,))])

#Download and load the training data

trainset = datasets.FashionMNIST("~/.pytorch/F_MNIST_data/", transform= transform, train=True,download =True)

testset = datasets.FashionMNIST('~/.pytorch/F_MNIST_data/', transform = transform, train= False, download =True)

In [0]:
trainloader = torch.utils.data.DataLoader(trainset, batch_size = 64, shuffle =True)

testloader = torch.utils.data.DataLoader(testset, batch_size = 64, shuffle=True)

In [0]:
from torch import nn, optim
import  torch.nn.functional as F

class Classifier(nn.Module):
  def __init__(self):
    super().__init__()
    self.fc1 = nn.Linear(784, 256)
    self.fc2 = nn.Linear(256,128)
    self.fc3 = nn.Linear(128,64)
    self.fc4 = nn.Linear(64,10)

  def forward(self, x):

    x = x.view(x.shape[0],-1)
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = F.relu(self.fc3(x))
    x = F.log_softmax(self.fc4(x), dim=1)
    return x


In [23]:
model = Classifier()

images, labels = next(iter(testloader))
# Get the class probabilities
ps = torch.exp(model(images))
# Make sure the shape is appropriate, we should get 10 class probabilities for 64 examples
print(ps.shape)

torch.Size([64, 10])


In [0]:
top_p,top_class = ps.topk(1, dim=1)

In [36]:
#look at the most likely cases for the first 10 examples
print(top_class[:5,:])

tensor([[9],
        [7],
        [7],
        [3],
        [7]])


In [0]:
equals = top_class ==labels.view(*top_class.shape)

In [44]:
equals

tensor([[ True],
        [False],
        [False],
        [False],
        [False],
        [False],
        [False],
        [False],
        [False],
        [False],
        [ True],
        [False],
        [ True],
        [False],
        [False],
        [False],
        [False],
        [False],
        [ True],
        [False],
        [False],
        [False],
        [False],
        [False],
        [False],
        [False],
        [False],
        [False],
        [False],
        [ True],
        [False],
        [False],
        [False],
        [False],
        [False],
        [False],
        [False],
        [ True],
        [False],
        [False],
        [False],
        [ True],
        [False],
        [ True],
        [False],
        [False],
        [False],
        [False],
        [False],
        [False],
        [False],
        [False],
        [False],
        [False],
        [False],
        [False],
        [False],
        [False],
        [False

In [49]:
accuracy  = torch.mean(equals.type(torch.FloatTensor))

print(f"Accuracy :{accuracy.item()*100}%")

Accuracy :12.5%


In [56]:
model = Classifier()
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr =0.003)

epochs =5
steps=0

train_losses,test_losses=[],[]
for e in range(epochs):
  running_loss =0
  for images,label in trainloader:

    optimizer.zero_grad()

    log_ps = model(images)

    loss = criterion(log_ps,label)

    loss.backward()

    optimizer.step()

    running_loss+=loss.item()

  else:
    test_loss =0
    accuracy =0

    #turn off gradients

    with torch.no_grad():
      for images,label in testloader:
        log_ps = model(images)

        test_loss += criterion(log_ps,label)

        ps = torch.exp(log_ps)
        top_p,top_class = ps.topk(1,dim=1)
        equals = top_class == label.view(*top_class.shape)
        accuracy+=torch.mean(equals.type(torch.FloatTensor))
    
      
    train_losses.append(running_loss/len(trainloader))
    test_losses.append(test_loss/len(testloader))

    print("Epoch: {}/{}.. ".format(e+1, epochs),
          "Training Loss: {:.3f}.. ".format(running_loss/len(trainloader)),
          "Test Loss: {:.3f}.. ".format(test_loss/len(testloader)),
          "Test Accuracy: {:.3f}".format(accuracy/len(testloader)))




Epoch: 1/5..  Training Loss: 2.142..  Test Loss: 1.724..  Test Accuracy: 0.492
Epoch: 2/5..  Training Loss: 1.217..  Test Loss: 0.931..  Test Accuracy: 0.697
Epoch: 3/5..  Training Loss: 0.803..  Test Loss: 0.735..  Test Accuracy: 0.737
Epoch: 4/5..  Training Loss: 0.682..  Test Loss: 0.663..  Test Accuracy: 0.758
Epoch: 5/5..  Training Loss: 0.619..  Test Loss: 0.617..  Test Accuracy: 0.772
